In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

# ColumnSelector
- 데이터 셋에서 특정 열을 선택하는 매우 단순한 기능
- make_pipelines를 활용할 수 있다

- 클래스원형
```
ColumnSelector(cols=None, drop_axis=False)
```

In [2]:
from sklearn.datasets import load_iris

# sklearn >= 2.0 포함
# 시험에서는 mlxtend를 사용해야 합니다.
from mlxtend.feature_selection import ColumnSelector

In [3]:
iris = load_iris()
x = iris.data
y = iris.target

In [11]:
x[:5, :]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [14]:
# 원본 데이터 셋에서 두 번째, 세 번째 컬럼만 선택
# 그냥 슬라이스 하거나, 배열 인덱싱을 사용해도 무방
selected_cols = ColumnSelector( cols=(1,2) )
selected_cols.transform(x)[:5, :]

array([[3.5, 1.4],
       [3. , 1.4],
       [3.2, 1.3],
       [3.1, 1.5],
       [3.6, 1.4]])

## 응용예제

In [20]:
# 모든 컬럼의 조합
from itertools import combinations

# 4개의 컬럼으로 만들 수 있는 모든 컬럼들의 조합을 찾는다.
comb = []
for size in range(1, 5):
  comb += list( combinations( range(4), r=size) )
print(comb)

[(0,), (1,), (2,), (3,), (0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3), (0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3), (0, 1, 2, 3)]


In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

In [24]:
pipe = make_pipeline(
  StandardScaler(), 
  ColumnSelector(), 
  KNeighborsClassifier()
)

# 그리드 서치를 통해서 모든 변수의 조합을 테스트해본다.
params = {
  'columnselector__cols': comb,
  'kneighborsclassifier__n_neighbors': range(1, 11)
}

search = GridSearchCV( pipe, params, cv=10 ).fit(x, y)

In [25]:
# 3번째, 4번째 컬럼을 이용해서 k가 1일때, 가장 좋은 점수를 내었다.
# k가 1이라면... 과적합의 가능성도 있습니다.
search.best_params_, search.best_score_

({'columnselector__cols': (2, 3), 'kneighborsclassifier__n_neighbors': 1},
 0.9733333333333334)

# ExhaustiveFeatureSelector
- 분류 및 회귀를 이용한 변수 선택 방법
- 클래스 원형
```
ExhaustiveFeatureSelector(
  estimator, min_features=1, max_features=1, 
  print_progress=True, scoring='accuracy', cv=5, 
  n_jobs=1, pre_dispatch='2n_jobs', clone_estimator=True, 
  fixed_features=None, feature_groups=None)
```

- 주요 파라미터
  - estimator: 변수 선택에 사용할 분류기 또는 회귀모델
  - min_features: 선택할 변수의 최소 개수, default=1
  - max_features: 선택할 변수의 최대 개수, default=1
  - scoring: 사용할 평가 방법, default=accuracy
    - 분류기: accuracy, f1, precision, recall, roc_auc
    - 회귀: mean_absolute_error, mean_squared_error, median_absolute_error, r2

## 예제

In [26]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

In [27]:
# 사용할 분류기 선택
knn = KNeighborsClassifier( n_neighbors=3 )

# 변수 선택기 객체 생성
efs = EFS(
  knn,
  min_features = 1,
  max_features = 4,
)

efs.fit(x, y)

Features: 15/15

ExhaustiveFeatureSelector(clone_estimator=True, cv=5,
                          estimator=KNeighborsClassifier(algorithm='auto',
                                                         leaf_size=30,
                                                         metric='minkowski',
                                                         metric_params=None,
                                                         n_jobs=None,
                                                         n_neighbors=3, p=2,
                                                         weights='uniform'),
                          max_features=4, min_features=1, n_jobs=1,
                          pre_dispatch='2*n_jobs', print_progress=True,
                          scoring='accuracy')

In [28]:
efs.best_score_, efs.best_feature_names_

(0.9733333333333334, ('0', '2', '3'))

# SequentialFeatureSelector
- 분류 및 회귀를 이용한 순차적 변수 선택
- 클래스 원형
```
SequentialFeatureSelector(
  estimator, k_features=1, forward=True, floating=False, verbose=0, score=None, cv=5, 
  n_jobs=1, pre_dispatch='2 n_jobs', clone_estimator=True, fixed_features=None, feature_groups= None )
```

- 주요 파라미터
  - estimator: 변수 선택에 사용할 분류기 또는 회귀
  - k_features: 선택할 변수의 수
    - 튜플인 경우 (최소개수, 최대개수) 사이에 가장 높은 점수를 받은 조합을 반환
  - forward: 전진선택(True), 후진제거(False)
  - floating: Step-wise(True)

## 예제

In [29]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [30]:
knn = KNeighborsClassifier( n_neighbors=4 )

sfs = SFS(
  knn, 
  k_features=(1, 4),
  forward=True,
  floating=False,
)
sfs.fit(x, y)

SequentialFeatureSelector(clone_estimator=True, cv=5,
                          estimator=KNeighborsClassifier(algorithm='auto',
                                                         leaf_size=30,
                                                         metric='minkowski',
                                                         metric_params=None,
                                                         n_jobs=None,
                                                         n_neighbors=4, p=2,
                                                         weights='uniform'),
                          floating=False, forward=True, k_features=(1, 4),
                          n_jobs=1, pre_dispatch='2*n_jobs', scoring=None,
                          verbose=0)

In [31]:
sfs.k_score_, sfs.k_feature_idx_

(0.9733333333333334, (0, 1, 2, 3))